<a href="https://colab.research.google.com/github/atick-faisal/Crowd-Emotion/blob/main/src_v4/CE_TE_MobileNet_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import os
import joblib
import numpy as np
import tensorflow as tf

from sklearn.utils import shuffle
from sklearn.metrics import classification_report, accuracy_score

tf.__version__

'2.4.1'

In [13]:
BASE_DIR         = '/content/drive/MyDrive/Research/Crowd Emotion v4/'
DATASET_DIR      = 'DatasetPC/'
FOLDS            = ['Fold 1', 'Fold 2', 'Fold 3', 'Fold 4', 'Fold 5']
EMOTIONS         = ['Approval', 'Disapproval', 'Neutral']

IMG_LEN          = 160
IMG_SIZE         = (IMG_LEN, IMG_LEN)
DATASET_LEN      = 14882

CONFIG           = 'LOG_SPEC_W400_H200_HAMM_IMG' + str(IMG_LEN)

TEST_FOLD        = 'Fold 1'
EPOCHS           = 10
LEARNING_RATE    = 0.001

In [14]:
path = os.path.join(BASE_DIR, DATASET_DIR)
X    = joblib.load(path + CONFIG + '_X.joblib')
y    = joblib.load(path + CONFIG + '_y.joblib')
f    = joblib.load(path + CONFIG + '_f.joblib')

In [15]:
# mask      = (f == FOLDS.index(TEST_FOLD))
# X_train   = X[~mask, :]
# X_test    = X[mask, :]
# y_train   = y[~mask, :]
# y_test    = y[mask, :]

In [16]:
X, y = shuffle(X, y)
# X_train     = X[:12000, :]
# y_train     = y[:12000, :]
# X_test      = X[12000:, :]
# y_test      = y[12000:, :]

In [17]:
preprocess_input = tf.keras.applications.mobilenet_v2.preprocess_input
rescale = tf.keras.layers.experimental.preprocessing.Rescaling(1./127.5, offset= -1)

In [18]:
IMG_SHAPE = IMG_SIZE + (3,)
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE, include_top=False, weights='imagenet')
base_model.trainable = False

In [19]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()
prediction_layer = tf.keras.layers.Dense(len(EMOTIONS))

In [20]:
def get_model():
    inputs = tf.keras.Input(shape=IMG_SHAPE)
    x = preprocess_input(inputs)
    x = base_model(x, training=False)
    x = global_average_layer(x)
    x = tf.keras.layers.Dropout(0.2)(x)
    outputs = prediction_layer(x)
    model = tf.keras.Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=LEARNING_RATE),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])
    return model

In [21]:
model = get_model()
# X_train, y_train = shuffle(X_train, y_train)
history = model.fit(X, y, epochs=EPOCHS)
model.save(os.path.join(BASE_DIR, 'TE_MOBILENET_ALL_DATA'))

Epoch 1/10
466/466 [==============================] - 17s 32ms/step - loss: 0.6594 - accuracy: 0.7092
Epoch 2/10
466/466 [==============================] - 15s 31ms/step - loss: 0.3749 - accuracy: 0.8493
Epoch 3/10
466/466 [==============================] - 15s 31ms/step - loss: 0.3409 - accuracy: 0.8636
Epoch 4/10
466/466 [==============================] - 15s 31ms/step - loss: 0.3051 - accuracy: 0.8810
Epoch 5/10
466/466 [==============================] - 15s 31ms/step - loss: 0.2973 - accuracy: 0.8820
Epoch 6/10
466/466 [==============================] - 15s 31ms/step - loss: 0.2781 - accuracy: 0.8869
Epoch 7/10
466/466 [==============================] - 15s 31ms/step - loss: 0.2710 - accuracy: 0.8924
Epoch 8/10
466/466 [==============================] - 15s 31ms/step - loss: 0.2694 - accuracy: 0.8903
Epoch 9/10
466/466 [==============================] - 15s 31ms/step - loss: 0.2663 - accuracy: 0.8889
Epoch 10/10
466/466 [==============================] - 15s 31ms/step - loss: 0.265

In [22]:
# prob = tf.keras.Sequential([model, tf.keras.layers.Softmax()])
# y_pred_hot = prob.predict(X_test)
# y_pred = np.argmax(y_pred_hot, axis=1)
# print(classification_report(y_test.ravel(), y_pred))